# Lecture 7: MCMC 
## Instructor： 胡传鹏（博士）[Dr. Hu Chuan-Peng]

### 南京师范大学心理学院[School of Psychology, Nanjing Normal University]

## Part 3: Re-cap of MCMC and popular algorithms

### 课前回顾: 

- 后验计算的问题
- 采样方法(蒙特卡洛方法)
- 马尔可夫链
- MCMC(马尔可夫蒙特卡洛方法)
- Metroplis-Hasting(MH)算法

- 后验计算的问题

$p(\theta|data) = \frac{p(data|\theta)p(\theta)}{\pmb {p(data)}}$

但是问题在于

1.求积分往往是很难的，并且有些积分可能没有解析解。
$P(data) =\int_{\theta}^{} p(data,\theta) =\int_{\theta}^{} p(data|\theta)p(\theta)d\theta$


2.若后验分布比较复杂，即使获得后验分布，也难以计算期望值。
$E(\theta) =\int_{\theta}^{} \theta p(\theta|data) d\theta$

- 采样方法(蒙特卡洛方法)

许多概率的问题非常复杂并且无法直接计算的。

但是我们可以通过大量模拟去接近真实的发生概率，即：

$p(发生的概率)=\frac{模拟情形下事件发生的次数}{模拟的次数}$

一个经典的例子是通过蒙特卡洛方法来求出不规则图形的面积。


![Image Name](https://cdn.kesci.com/upload/image/rk6x6gx69n.png?imageView2/0/w/640/h/640)

该方法的一个具体应用为 **接受-拒绝采样**

为了求解一个不常见的分布$p(x) = 0.3*e^{-(x-0.3)^2} + 0.7*e^{-(x-2)^{2/0.3}}/1.21$。

1. 我们可以首先在一个常见的正态分布q(x)中进行采样进行采样，往往称为**建议分布(proposal distribution)**。
2. 如果该x对应的p(x)离q(x)距离近，即两个函数的y值接近，我们就接受该x的采样，否则拒绝该采样。

  我们通过 $\frac{p(θ)}{q(θ)} \ge U$ 判断是否接受该参数值

3. 这样反复采样在进行拒绝的过程，可以使得最后的采样样本接近 p(x) 分布。


![Image Name](https://cdn.kesci.com/upload/image/rk6x6xprcw.png?imageView2/0/w/640/h/640)

- 马尔可夫链

马尔科夫链假设某一时刻状态转移的概率只依赖于它的前一个状态。

$P(X_{t+1}|...X_{t−2},X_{t−1},X_{t})=P(X_{t+1}|X_{t})$

上述公式可以表示为状态转移矩阵：

$\begin{bmatrix}
     &  陆地 & 海洋\\
陆地& 0.75 & 0.25\\
海洋 & 0.1 & 0.9\\ 
\end{bmatrix}$

马尔科夫链的**特征**在于：

无论初始状态如何，只要状态转移一直持续下去，处在每个状态的概率会服从一个平稳分布。


![Image Name](https://cdn.kesci.com/upload/image/rk6x7auvhk.png?imageView2/0/w/640/h/640)

- MCMC(马尔可夫蒙特卡洛方法)

结合了以下特征：
1. 蒙特卡洛方法中的采样思想；
2. 接受拒绝采样中的接受率；
3. 马尔可夫链中状态依赖与状态转移后形成平稳分布；

MCMC方法结合了很多优秀思想与特性，使得我们可以从一个简单的分布中采样，然后不断地接近复杂地目标分布。

- Metroplis-Hasting(MH)算法

MCMC不是一个具体的采样算法，而是一类算法簇。这类算法都具有上述特征。

Metroplis-Hasting 算法是MCMC当中的一种经典实现。

MH的特点在于扩展了接受拒绝采样法，主要体现为：
1. 利用马尔科夫链的性质，从上一次采样相关的建议分布中进行采样，
   - 即从随机采样 `proposal_theta = st.norm(0.5,0.1).rvs(1)` 变为 `theta_t1 = st.norm(theta_t0,0.1).rvs(1)`
2. 通过两次采样的后验比值去判断是否接受参数，而不是和随机分布进行比较。
   - 即$\frac{p(theta)}{ q(theta)} \ge U$ 转变为 $\frac{p(theta_{t1})}{ p(theta_{t0})} \ge U$。

MH算法虽然比接受拒绝采样多了诸多优点，其仍然存在不足：
1. proposal distribution 的标准差太大会导致大量被拒绝的样本，而太小的标准差会导致花费更长时间来“探索分布”；

  `proposal_theta1 = st.norm(proposal_theta0,0.5).rvs(1)` 比如将建议分布的标准差从0.5修改为5

2. 当模型参数很多的时候，MH 采样的效率非常低，因为它的采样是根据 proposal distribution进行随机选取的；
3. 对于高维后验分布，MH 的采样是非常不均匀的。

### 实践中的MCMC

通过MH算法，我们了解了MCMC的诸多特性。

但是在实际数据分析中，仅使用MH算法往往会因其缺点而遇到各种问题。

因此，在实际操作中，更常用的是以下MCMC算法，包括：
- 吉布斯采样 (Gibbs sampling)
- 哈密顿蒙特卡洛(Hamiltonian Monte Carlo, HMC)
- 差分进化算法 (Differential evolution) DE-MCMC

MCMC采样的一个难题是，如果后验分布函数 $p(\theta)$ 中存在多个参数 $\theta$ 需要估计该怎么办？

比如，在一个有两个参数(维度)的后验分布中，将两个参数形成的参数对与其似然或后验一起，形成了一个三维空间；

例如，图中的灰黑色等高线是后验的负数 $-p(\theta_{i})$，

蓝色线条所在平面代表两个参数 $\theta_{1}$, $\theta_{2}$。

![Image Name](https://cdn.kesci.com/upload/image/rjvgyvcsre.gif?imageView2/0/w/640/h/640)

绿色为接受的采样，红色是拒绝的采样。

根据MH算法，我们需要从多个参数的联合分布 $p(\theta_{1},\theta_{2})$ 中进行采样。

即，每次我们从蓝色线条所在平面采样一个点，都会得到其两个维度的值，这两个值对应了$\theta_{1}$, $\theta_{2}$。

MH的问题在于：
- 如果参数维度大于二，比如模型有n个参数，那么每次采样一个点，就对应n个参数$\theta_{n}$。
- 此时的联合分布也非常复杂，$p(\theta_{1},\theta_{2}, ..., \theta_{n})$。

这样的结果是，采样受到维度的影响，随机性增加了，这会导致拒绝参数的概率增加，比如上图中红的点为MH算法拒绝的参数。

**吉布斯采样 (Gibbs sampling)**

为解决多参数采样的问题，Gibbs算法提供了一个非常简单的思路。

在对一个参数进行采样时，固定其他参数：

- 比如，对于二维参数，Gibbs采样首先对 $\theta_{1} \sim p(\theta_{1})$ 进行采样
- 之后固定 $\theta_{1}$, 对$\theta_{2} \sim p(\theta_{2}|\theta_{1})$ 进行采样
- 其**特点**在于将对联合分布的采样转化为条件分布，即从 **$p(\theta_{2},\theta_{1})$ 变为 $p(\theta_{2}|\theta_{1})$**

使用Gibbs sampling对二元高斯分布进行采样

![Image Name](http://gorayni.github.io/assets/posts/gibbs/gibbs2.gif)

Gibbs采样可以看作是 MH 算法的一个**特例**。因为它只是减少了MH计算的复杂度。

此外，Gibbs采样不是对一组参数 $\theta_{n}$ 进行接受和拒绝的判断，而是分别对 $\theta_{1},\theta_{2}, ..., \theta_{n}$ 进行接受和拒绝的判断，这提高了接受参数的概率。

Gibbs采样的问题在于，这种简化存在代价：
- 参数间要满足条件分布的假设，不满足假设可能导致采样时拒绝率会很高，或者采样分布无法近似后验分布。
- 对条件分布的采样可以提高采样的效率(相对于MH)，但同样会受到多峰分布的影响。

**哈密顿蒙特卡洛(Hamiltonian Monte Carlo, HMC)**

为解决 MH 中 proposal distribution 由于随机采样导致效率低的问题而开发。

HMC结合物理概念来优化 MCMC：
- HMC 通过基本物理规律 **能量=势能+动能** 假设采样的分布(后验分布)为系统的总能量，然后将该系统分为两个子部分
- 由于动量增加，势能就会减少，因此**系统总能量不会变化**。这说明我们采样的分布不会因为系统的动态变化而变化，这类似于马尔科夫的平稳分布

- 势能，与位置有关，比如对于重力势能，高度越高，势能越大。
  
  相似的，可以**把参数的位置看作高度，把参数对应的似然看作势能**。
	
	在拒绝采样中，似然值越大对应的参数越有可能被接受，因此，这里势能越大的参数也越有可能被接受。

- 动能：与速度有关，比如动能公式 $\frac{1}{2}mv^2$。
  
在MCMC中，其与似然函数的梯度相对应。
	
梯度指向了似然最大的地方，即图中的波谷(这里的灰黑色等高线是后验的负数 $-p(\theta_{i})$
	
	
![Image Name](https://cdn.kesci.com/upload/image/rk931ktu8i.jpg?imageView2/0/w/640/h/640)

Source: https://regenerativetoday.com/machine-learning-gradient-descent-concept/


从图中可以发现，沿着梯度进行采样可以更多的采样到波谷位置的参数。这比MH算法根据建议分布进行随机采样的效率更高。

![Image Name](https://cdn.kesci.com/upload/image/rjvh3zx4an.gif?imageView2/0/w/640/h/640)

红色的线条是 HMC 表示沿着梯度进行采样。

绿色的点表示接受的采样，红色的点表示被拒绝的采样。

从图中可以看到，HMC算法几乎没有被拒绝的采样，其采样效率高。

HMC的特点总结：
- 结合 **能量=势能+动能** 的物理特性
- 其中，势能与MH计算似然进行接受拒绝类似
- 动能为HMC算法独特的一部分，即需要额外计算梯度
- 通过梯度进行采样，能凭借更少的样本接近后验

HMC算法的缺点
- 如果无法计算后验分布函数的梯度，那么就无法使用HMC
- 计算梯度和计算似然都很花费算力，但是相比于MH和Gibbs，HMC所需的采样数量更少

**差分进化算法 (Differential evolution) DE-MCMC**

当参数太多时(比如多于20个)：
- 尽管Gibbs采样可以从条件分布进行采样，但参数越多，需要求解的条件概率越多，这导致采样方法变慢。
- 对于HMC来说，参数太多时梯度计算也非常困难。

差分进化算法的目的在于：通过生物系统中的**遗传进化算法**解决多参数采样的问题。

其特点在于：
- 把多个参数**采样**视作**生物种群**，种群中每个个体的**基因特征**视作**参数维度**
- 生物种群会随着迭代进行迁移
- 个体会随着迭代产生基因突变

**生物种群与个体**

相较于其他算法每次从建议分布中抽取1个参数样本，DE算法会一次性从建议分布中抽取多个参数样本。

这多个参数样本形成了一个种群。

由于每个参数样本包含多个参数维度，这些不同参数维度构成了每个样本的基因特征。

$x_{i}$ 为种群中的个体, i属于1到n。

> $x_{i}$ 也被称为不同的chain。
> plus，相较于其他算法只有4-5chain，DE的chain是他们的数10倍。

$theta_{j}$ 为个体的“基因”, $j \in [1, m]$。

|              | x1  | x2  | x3  | ... | xn  | 
| -----        | --- | --- | --- | --- | --- | 
| $\theta_{1}$ | 5   | 2   | 5  | ... | 3   |
| $\theta_{2}$ | 3   | 3   | 4  |  ... | 2   |
| $\theta_{3}$ | 1   | 2   | 4  | ... | 1   |
| ...          | ... | ... | ... | ... |  ...|  
| $\theta_{m}$ | 4   | 5   | 4  |  ... | 5    |

**基因变异与MCMC的采样**

DE的最重要的特点为，每个个体(chain)的基因(参数维度)可以突变，对应MCMC的一次proposal。

具体表示为： $v_{i}=x_{r 1}+\lambda *\left(x_{r 2}-x_{r 3}\right)$

突变分为三个步骤：
1. 随机从种群中选择三个个体, 即上面的三列数据。用 $x_{r 1}$, $x_{r 2}$ 和 $x_{r 3}$ 表示。
2. 突变的差异为 $x_{r 2}-x_{r 3}$ 两个向量的差，如下图。
3. 这个突变的差异会施加给第三个个体 $x_{r 1}$, 形成新的一代个体 $v_{1}$

如同生物进化一样，每个新的个体 $v_{1}$ 包含了父代 $x_{r 1}$ 的基因，也允许了已经基因的变化。

> plus, 其中 $\lambda$ 是突变系数，控制突变的大小。

![Image Name](https://matteding.github.io/images/diff_evol.gif)

**种群的迁移**

由于每次基因突变意味着新的个体产生，即新的种群 $v_{i}$

|              | v1  | v2  | v3  | ... | vn  | 
| -----        | --- | --- | --- | --- | --- | 
| $\theta_{1}$ |  2  | 2   | 3  | ... | 3   |
| $\theta_{2}$ | 3   | 6   | 3  |  ... | 4   |
| $\theta_{3}$ | 1   | 2   | 1  | ... | 1   |
| ...          | ... | ... | ... | ... |  ...|  
| $\theta_{m}$ | 3   | 5   | 3  |  ... | 4    |

而在生物进化中，新生的个体不意味着一定存活（即MCMC中的一次proposal不一定被接受），而父辈个体也可能还没有死亡。

因此我们允许上一代老的种群 $x_{i}$ 和新的种群 $v_{i}$ 同时存在。

但我们会保持种群个体数n不变。因此我们会在 $v_{i}$ 中随机挑选一些个体，用 $x_{i}$ 的个体进行替换。

DE-MCMC算法总结：
- DE算法融合了生物进化的特性
- 其中，基因变异部分的向量变化和HMC算法中的梯度类似
- 迁移部分是DE算法更加独特的一部分，它将不同的chain进行混合，增加了采样的随机性

DE-MCMC算法的缺点：
- 需要大量的个体(chain)，因此计算量大
- 当参数太少，采样的效率低，并且没有其他算法对后验的近似度高

**三种算法的对比：**


|          | Gibbs                            | HMC            | DE                     |
| -------- | -------------------------------- | -------------- | ---------------------- |
| 优点     | 用于处理多参数的情景, 速度相对较快         | 采样效率高         | 适用于参数非常多的情况 |
| 特点     | 与 MH 算法类似，对条件分布进行采样         | 结合了物理特性，考虑了分布函数梯度 | 结合了生物遗传特性         |
| 缺点     | 需要较多采样样本，需满足条件分布的假设 | 需要额外计算梯度   | 需要很多chain，计算量大   |
| 代表工具 | R-JAGS                           | Rstan 和 pymc  | pymc                   |

HMC是现代最流行的算法，其变种 No-U-Turn sampler（NUTS）方法是 pymc 和 Stan 默认使用算法。


![Image Name](https://cdn.kesci.com/upload/image/rjvix0jc6b.png?imageView2/0/w/320/h/320)


![Image Name](https://cdn.kesci.com/upload/image/rjviyt1d5d.png?imageView2/0/w/320/h/320)